# JAX的自动向量化

> 作者：Matteo Hessel

在上一节我们讨论通过 `jax.jit`函数进行的JIT编译。本届讨论了JAX中的另一种转换，即通过`jax.vmap`进行矢量化。

## 手动向量化

让我们来看下面的简单代码，该代码计算两个一维向量的卷积：

In [1]:
import jax
import jax.numpy as jnp

x = jnp.arange(5)
w = jnp.array([2., 3., 4.])

def convolve(x, w):
    output = []
    for i in range(1, len(x)-1):
        output.append(jnp.dot(x[i-1:i+2], w))
    return jnp.array(output)

convolve(x, w)

DeviceArray([11., 20., 29.], dtype=float32)

假设我们想要将此函数应用于一批权重`w`和一批向量`x`上：

In [2]:
xs = jnp.stack([x, x])
ws = jnp.stack([w, w])

最简单的想法就是在Python中循环遍历该批处理：

In [3]:
def manually_batched_convolve(xs, ws):
    output = []
    for i in range(xs.shape[0]):
        output.append(convolve(xs[i], ws[i]))
    return jnp.stack(output)

manually_batched_convolve(xs, ws)

DeviceArray([[11., 20., 29.],
             [11., 20., 29.]], dtype=float32)

这虽然会产生正确的结果，但是效率不高。

为了有效地批处理计算，通常必须手动重写函数以确保它以向量的形式完成。这并不是特别难实现，但确实设计更改函数如何处理索引，维度和输入的其他部分。

例如，我们可以手动重写`convolve()`来支持跨批处理维度的向量化计算，如下所示：

In [4]:
def manually_vectorized_convolve(xs, ws):
    output = []
    for i in range(1, xs.shape[-1]-1):
        output.append(jnp.sum(xs[:, i-1:i+2] * ws, axis=1))
    return jnp.stack(output, axis=1)

manually_vectorized_convolve(xs, ws)

DeviceArray([[11., 20., 29.],
             [11., 20., 29.]], dtype=float32)

这种重新实现是混乱且容易出错的。 幸运的是，JAX提供了另一种方法。

## 自动向量化

在JAX中，`jax.vmap`转换旨在自动生成函数的向量化实现：

In [5]:
auto_batch_convolve = jax.vmap(convolve)
auto_batch_convolve(xs, ws)

DeviceArray([[11., 20., 29.],
             [11., 20., 29.]], dtype=float32)

它通过类似于`jax.jit`的函数追踪功能，并在每个输入的开头自动添加批处理来实现此目的。

如果批次维度不是第一个，则可以使用`in_axes`和`out_axes`参数指定批次维度在输入和输出中的位置。如果批处理轴对于所有输入和输出（或列表）相同，则这些值可以是整数。

In [6]:
auto_batch_convolve_v2 = jax.vmap(convolve, in_axes=1, out_axes=1)

xst = jnp.transpose(xs)
wst = jnp.transpose(ws)

auto_batch_convolve_v2(xst, wst)

DeviceArray([[11., 11.],
             [20., 20.],
             [29., 29.]], dtype=float32)

`jax.vmap`还支持仅对其中一个参数进行批处理的情况：例如，如果您想将一组权重`w`与一组向量`x`进行卷积，则在这种情况下，可以将`in_axes`参数设置为`None`：

In [ ]:
batch_convolve_v3 = jax.vmap(convolve, in_axes=[0, None])

batch_convolve_v3